In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
#import numpy as np
from scipy.stats import skew, kurtosis
from scipy import signal
from scipy.fft import fft
from biosppy.signals import ecg  # Biosppy is a library for biosignal processing
import pywt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the CSV file into a DataFrame
csv_path = 'D:\Omar\Friends\European_HealthCare_Hackathon\ecg_hospitalization\data\processed\meta\data_pairs.csv'  # Replace with the path to your CSV file
df = pd.read_csv(csv_path)

In [3]:
# Initialize empty lists to store data
data_list = []
labels_list = []

In [4]:
# Iterate through rows and load .npy files
for index, row in tqdm(df.iterrows()):
    file_path = row['np_file_path']
    label = row['label']

    # Load the .npy file
    loaded_data = np.load(file_path)

    # Append the loaded data and label to the lists
    data_list.append(loaded_data)
    labels_list.append(label)

23292it [09:11, 42.23it/s] 


In [8]:
# Convert lists to NumPy arrays
data_array = np.array(data_list)
labels_array = np.array(labels_list)

In [9]:
# Print shapes for verification
print("Data Array Shape:", data_array.shape)
print("Labels Array Shape:", labels_array.shape)

Data Array Shape: (23292, 8, 5000)
Labels Array Shape: (23292,)


In [10]:
# Load the CSV file into a DataFrame
#csv_path = 'your_file.csv'  # Replace with the path to your CSV file
#df = pd.read_csv(csv_path, header=None, names=['np_file_path', 'label'])

In [25]:
indices_with_nans = [370, 4733, 4936, 5404, 8354, 9146, 10268, 10879, 12946, 14674, 15413, 15702]

In [26]:
data_array = data_array[np.logical_not(np.isin(np.arange(len(data_array)), indices_with_nans))]

In [27]:
labels_array = labels_array[np.logical_not(np.isin(np.arange(len(labels_array)), indices_with_nans))]

In [11]:
# Assuming ecg_data is your 3D array of raw ECG data
# Assuming labels are integers (0, 1, 2) representing the three classes
num_patients, num_leads, num_time_points = data_array.shape

In [13]:
#import pickle

In [14]:
# Open the file in binary read mode ('rb') to unpickle the data
#with open('batch_of_data.pickle', 'rb') as file:
#    loaded__data = pickle.load(file)

In [15]:
#ecg_data = []
#for patient in loaded_data:
#    new_patient_format = np.array(list(patient.values()))
#    ecg_data.append(new_patient_format)
#    print(len(new_patient_format[7]))

In [16]:
#ecg_data = np.array(ecg_data)

In [17]:
ecg_data = data_array
labels = labels_array

In [18]:
#num_patients = 23292
#num_leads = 8
#num_time_points = 5000

In [19]:
#np.random.seed(42)
#ecg_data = np.random.randn(num_patients, num_leads, num_time_points)
#labels = np.random.randint(0, 3, size=num_patients)

In [21]:
# Function to extract features from each lead
def extract_features(lead):
    # 1. Statistical Features
    mean_value = np.mean(lead)
#    print('mean_value :', mean_value)
    median_value = np.median(lead)
#    print('median_value :', median_value)
    std_dev_value = np.std(lead)
#    print('std_dev_value :', std_dev_value)
    skewness_value = skew(lead)
#    print('skewness_value :', skewness_value)
    kurtosis_value = kurtosis(lead)
#    print('kurtosis_value :', kurtosis_value)

    # 2. Time-Domain Features
    # You might need to preprocess the data to find R-peaks for RR interval calculations
    # Example using biosppy
    rpeaks = ecg.ecg(lead, sampling_rate=500, show=False)['rpeaks']
   # _, rpeaks = ecg.ecg(lead, sampling_rate=500, show=False)
#    print("YES")
    rr_interval = np.diff(rpeaks)

    # Calculate features from RR intervals
    rr_mean = np.mean(rr_interval)
#    print('rr_mean :', rr_mean)    
    heart_rate = 60 / rr_mean

    # 3. Frequency-Domain Features
    #power_spectral_density (psd) shape is (num_time_points // 2 + 1)
    f, psd = signal.welch(lead, fs=500)
    dominant_frequency = f[np.argmax(psd)]
#    print('dominant_frequency :', dominant_frequency)
    spectral_entropy = -np.sum(psd * np.log2(psd + 1e-10))
#    print('spectral_entropy :', spectral_entropy)
    
    # 4. Wavelet Transform (using PyWavelets library)
    # Wavelet Transform Features
#    coeffs = pywt.wavedec(lead, 'db1', level=4)
#    print('coeffs :', coeffs)    

    # Combine all features into a single array
    extracted_features = np.array([
        mean_value, median_value, std_dev_value, skewness_value, kurtosis_value,
        rr_mean, heart_rate, dominant_frequency, spectral_entropy
#        *coeffs[0], *coeffs[1], *coeffs[2], *coeffs[3]
    ])

    return extracted_features

#    (cA, cD) = pywt.dwt(lead, 'db1')

    # 5. Heart Rate Variability Features
    # Already calculated RR intervals, you can extract various features from them

    # 6. Dynamical Features
    # You might need a dynamic model or use dynamic time warping techniques

    # 7. Deep Learning Representations
    # Use pre-trained models or train your own CNN/RNN on ECG data

    # 8. Principal Component Analysis (PCA)
#    from sklearn.decomposition import PCA

#    flattened_data = np.reshape(ecg_data, (num_patients, -1))
#    pca = PCA(n_components=10)  # Adjust the number of components
#    pca_features = pca.fit_transform(flattened_data)
    
    # 9. Cross-Lead Features
#    lead_correlations = np.zeros((num_patients, num_leads, num_leads))
#    for i in range(num_patients):
#        for j in range(num_leads):
#            for k in range(num_leads):
#                lead_correlations[i, j, k] = np.corrcoef(ecg_data[i, j, :], ecg_data[i, k, :])[0, 1]

In [22]:
num_features = len(extract_features(ecg_data[0, 0, :]))
print(num_features)

9


In [23]:
# Initialize an empty array to store features for each patient
num_features = len(extract_features(ecg_data[0, 0, :]))
patient_features = np.zeros((num_patients, num_features * num_leads))

In [35]:
# Apply feature extraction for each lead and concatenate features for each patient
for patient_index in tqdm(range(23204, num_patients)):
    if(patient_index in [370, 899, 4733, 4936, 5404, 8354, 9146, 9560, 10268, 10879, 11915, 12946, 13441, 14674, 15413, 15702, 16190, 22258, 23204]):
        patient_features[patient_index, :] = np.zeros((num_leads, num_features)).flatten()
        continue
    patient_lead_features = np.zeros((num_leads, num_features))
    
    for lead_index in range(num_leads):
        patient_lead_features[lead_index, :] = extract_features(ecg_data[patient_index, lead_index, :])
    
    # Concatenate features for the current patient
    patient_features[patient_index, :] = patient_lead_features.flatten()

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [00:06<00:00, 14.55it/s]


In [37]:
patient_features.shape

(23292, 72)

In [38]:
y = labels

In [39]:
y.shape

(23292,)

In [42]:
# Assuming patient_features is your 2D array of features (patients x features)
# Assuming y is your target labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(patient_features, y, test_size=0.2, random_state=42)

In [43]:
#from sklearn.utils.class_weight import compute_sample_weight

In [44]:
unique_classes = np.unique(y_train)

In [45]:
unique_classes

array([0, 1, 2])

In [40]:
#class_weights = compute_sample_weight(class_weight='balanced', y=y_train)

In [41]:
#class_weights

In [42]:
#class_weights.shape

In [46]:
patient_features.shape

(23292, 72)

In [47]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
# Class proportions
p1 = 0.61
p2 = 0.29
p3 = 0.1

In [52]:
# Calculate weights
w1 = 1 / p1
w2 = 1 / p2 + 0.2
w3 = 1 / p3 + 1 

In [53]:
# Normalize weights
sum_weights = w1 + w2 + w3
normalized_w1 = w1 / sum_weights
normalized_w2 = w2 / sum_weights
normalized_w3 = w3 / sum_weights

In [59]:
# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(class_weight = {0:0.0001, 1:0.0009, 2:1-0.001}, n_estimators=200, random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

RandomForestClassifier(class_weight={0: 0.0001, 1: 0.0009, 2: 0.999},
                       n_estimators=200, random_state=42)

In [60]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_scaled)

In [61]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [62]:
print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.40673964370036486
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.53      0.57      2839
           1       0.62      0.14      0.23      1366
           2       0.11      0.45      0.17       454

    accuracy                           0.41      4659
   macro avg       0.45      0.37      0.32      4659
weighted avg       0.57      0.41      0.43      4659



In [58]:
len(y)

23292

In [ ]:
count = np.count_nonzero(y == 0)
count / len(y)

In [ ]:
count = np.count_nonzero(y == 1)
count / len(y)

In [ ]:
count = np.count_nonzero(y == 2)
count / len(y)